In [1]:
import pandas as pd
from pathlib import Path


# helpers 

In [2]:
src_path = Path('/mnt/c/Users/umarh/Dropbox/Work/Employment/Shopify Transformation - Nabil')

In [3]:
raw = src_path.joinpath('raw_data')
dim = src_path.joinpath('dimension_lookups')
output = src_path.joinpath('output')

In [5]:
def get_newest_file(path : Path, ext : str) -> Path:
    
    file_d = {
                  f : 
                  pd.to_datetime(f.stat().st_mtime, unit='s') 
              for f 
              in path.glob(f'*.{ext}')
             }
    return max(file_d, key =file_d.get) 

# Sample Output



In [7]:
samp = pd.read_csv(src_path.joinpath('Result.csv'))

# Raw Data

In [58]:
raw_df = pd.read_excel(get_newest_file(raw, 'xls'))

In [59]:
# remove total column, has chinese character in so will use interger slicing. 
raw_df = raw_df.iloc[:,:-1]

In [60]:
# chinese char again so rename using list slicing.
raw_df = raw_df.rename(columns={raw_df.columns[0] : 'src_sku'})

In [61]:
raw_transform = pd.melt(raw_df,
        id_vars='src_sku',
        var_name='Option1 Value', 
        value_name='70 rue de la prulay').dropna(subset=['70 rue de la prulay']
        )

In [62]:
raw_transform['SKU'] = (raw_transform['src_sku'].str.replace('\(.*\)','',regex=True) + '-' + raw_transform['Option1 Value']).str.strip()

In [63]:
raw_transform['SKU'] = raw_transform['SKU'].str.replace('-F', '',regex=False)

# Lookups 

In [39]:
dim_df = pd.read_excel(get_newest_file(dim,'xlsx'),engine='openpyxl')

In [134]:
# don't need this now, join logic is sku + '-' + size 
# dim_df['SKU_Cleaned'] = dim_df['SKU'].str.split('-',expand=True)[0]

In [135]:
# dim_df_sku_handl = dim_df.drop_duplicates(subset=['SKU_Cleaned', 'Handle'])[['Handle','SKU_Cleaned']]

In [136]:
# dim_df_sku_handl = dim_df_sku_handl.rename(columns={'SKU_Cleaned': 'SKU'})

In [83]:
result = pd.merge(raw_transform.drop('Option1 Value',axis=1),
         dim_df,
         on=['SKU'],
         
        how='left')

In [91]:
result['Handle'] = result['Handle'].fillna(result.groupby('src_sku')['Handle'].ffill().bfill())

In [100]:
result[result['src_sku'].eq('CA006')]

,src_sku,70 rue de la prulay,SKU,Handle,Option1 Name,Option1 Value
112,CA006,11.0,CA006-XS,gothic-patterned-flocking-flared-sleeves-sexy-...,NaN,NaN
328,CA006,11.0,CA006-S,punk-asymmetrical-women-broken-holes-black-hoo...,Size,S
806,CA006,21.0,CA006-M,punk-asymmetrical-women-broken-holes-black-hoo...,Size,M
1286,CA006,11.0,CA006-L,punk-asymmetrical-women-broken-holes-black-hoo...,Size,L
1793,CA006,15.0,CA006-XL,punk-asymmetrical-women-broken-holes-black-hoo...,Size,XL
2242,CA006,7.0,CA006-2XL,punk-asymmetrical-women-broken-holes-black-hoo...,Size,2XL
2824,CA006,7.0,CA006-3XL,punk-asymmetrical-women-broken-holes-black-hoo...,Size,3XL


In [69]:
#assign missing columns
result = result.assign(**{s : pd.NA for s in samp.columns if not s in result.columns})

In [70]:
raw_transform.shape

(3619, 4)

In [71]:
result_final = result[samp.columns]

In [170]:
pd.Timestamp('today').strftime('%Y%m%d_result.csv')

'20210911_result.csv'

In [72]:
result_final = result_final.sort_values(['SKU'])

In [75]:
missing_df = result_final[result_final['Handle'].isna()==True]

In [78]:
dt = pd.Timestamp('today').strftime('%Y%m%d_%H%M%S')

In [79]:
result_final.dropna(subset=['Handle']).to_csv(
    output.joinpath(f"{dt}_result.csv"), index=False)

# Output Missing SKUs

In [81]:
missing_df.to_csv(src_path.joinpath('missing_skus',f"{dt}_missing_sku.csv"),index=False)

In [82]:
missing_df['SKU']

,Handle,SKU,Option1 Name,Option1 Value,Option2 Name,Option2 Value,Option3 Name,Option3 Value,70 rue de la prulay
30,NaN,AS07301,NaN,NaN,<NA>,<NA>,<NA>,<NA>,28.0
31,NaN,AS07302,NaN,NaN,<NA>,<NA>,<NA>,<NA>,27.0
36,NaN,AS08001,NaN,NaN,<NA>,<NA>,<NA>,<NA>,10.0
37,NaN,AS08002,NaN,NaN,<NA>,<NA>,<NA>,<NA>,19.0
39,NaN,AS083,NaN,NaN,<NA>,<NA>,<NA>,<NA>,19.0
...,...,...,...,...,...,...,...,...,...
1737,NaN,WT064-L,NaN,NaN,<NA>,<NA>,<NA>,<NA>,5.0
1225,NaN,WT064-M,NaN,NaN,<NA>,<NA>,<NA>,<NA>,5.0
758,NaN,WT064-S,NaN,NaN,<NA>,<NA>,<NA>,<NA>,5.0
2196,NaN,WT064-XL,NaN,NaN,<NA>,<NA>,<NA>,<NA>,4.0
